In [38]:
# Importing modules
import os
import sqlalchemy
SQLALCHEMY_SILENCE_UBER_WARNING = 1
import pandas as pd

In [39]:
# Defining database connection
def connect_tcp_socket() -> sqlalchemy.engine.base.Engine:
    """ Initializes a TCP connection pool for a Cloud SQL instance of MySQL. """
    db_host = "34.132.33.93" # e.g. '34.132.33.93' ('172.17.0.1' if deployed to GAE Flex)
    db_user = "root" # e.g. 'my-db-user'
    db_pass = "Pandas2020!" # e.g. 'my-db-password'
    db_name = "database" # e.g. 'my-database'
    db_port = 3306 # e.g. 3306

    engine = sqlalchemy.create_engine(
        sqlalchemy.engine.url.URL.create(
            drivername="mysql+pymysql",
            username=db_user,
            password=db_pass,
            host=db_host,
            port=db_port,
            database=db_name,
        ),
    )
    return engine

engine = connect_tcp_socket() 
conn = engine.connect()

In [51]:
result = conn.execute("SHOW TABLES;").fetchall()
for r in result:
    print(r) 

('articles',)
('customers',)
('kpi_average_order_value',)
('kpi_conversion_rate',)
('kpi_customer_retention',)
('kpi_fashion_news_effectiveness',)
('kpi_fashion_news_frequency',)
('kpi_inventory_turnover',)
('kpi_product_sales',)
('kpi_product_sales_perc',)
('kpi_purchase_frequency',)
('kpi_sales_growth',)
('transactions',)


# Querying all "transactions" data for usage in pandas

In [4]:
transactions_header_sql = """
    SELECT COLUMN_NAME
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = 'transactions'
    ORDER BY ORDINAL_POSITION
"""

transactions_header = pd.read_sql(con = conn, sql = transactions_header_sql)
transactions_header.head()

,COLUMN_NAME
0,date
1,customer_id
2,article_id
3,price
4,sales_channel_id


In [3]:
# transactions = pd.read_csv('/Users/maxheilingbrunner/Documents/Capstone/data/transactions.csv', delimiter = ",")

transactions_sql = """
    SELECT *
    FROM transactions;
"""

transactions = pd.read_sql(con = conn, sql = transactions_sql)
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [4]:
# Renaming date column
transactions.rename(columns = {'t_dat':'date'}, inplace = True)

In [5]:
# Changing dates to datetimes
transactions["date"] = \
    pd.to_datetime(transactions["date"])

In [56]:
transactions.dtypes

date                datetime64[ns]
customer_id                 object
article_id                   int64
price                      float64
sales_channel_id             int64
dtype: object

# Querying all "customers" data for usage in pandas

In [6]:
customers_header_sql = """
    SELECT COLUMN_NAME
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = 'customers'
    ORDER BY ORDINAL_POSITION
"""

customers_header = pd.read_sql(con = conn, sql = customers_header_sql)
customers_header.head()

,COLUMN_NAME
0,customer_id
1,fashion_news
2,Active
3,club_member_status
4,fashion_news_frequency


In [7]:
customers_sql = """
    SELECT *
    FROM customers
"""

customers = pd.read_sql(con = conn, sql = customers_sql)
customers.head()

,customer_id,fashion_news,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0,0,ACTIVE,NONE,49,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0,0,ACTIVE,NONE,25,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0,0,ACTIVE,NONE,24,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0,0,ACTIVE,NONE,54,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1,1,ACTIVE,Regularly,52,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [62]:
customers.dtypes

customer_id               object
fashion_news               int64
Active                     int64
club_member_status        object
fashion_news_frequency    object
age                        int64
postal_code               object
dtype: object

# Querying all "articles" data for usage in pandas

In [8]:
articles_header_sql = """
    SELECT COLUMN_NAME
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = 'articles'
    ORDER BY ORDINAL_POSITION
"""

articles_header = pd.read_sql(con = conn, sql = articles_header_sql)
articles_header.head()

,COLUMN_NAME
0,article_id
1,product_code
2,prod_name
3,product_type_no
4,product_type_name


In [9]:
articles_sql = """
    SELECT *
    FROM articles
"""

articles = pd.read_sql(con = conn, sql = articles_sql)
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [15]:
articles.dtypes

article_id                       int64
product_code                     int64
prod_name                       object
product_type_no                  int64
product_type_name               object
product_group_name              object
graphical_appearance_no          int64
graphical_appearance_name       object
colour_group_code                int64
colour_group_name               object
perceived_colour_value_id        int64
perceived_colour_value_name     object
perceived_colour_master_id       int64
perceived_colour_master_name    object
department_no                    int64
department_name                 object
index_code                      object
index_name                      object
index_group_no                   int64
index_group_name                object
section_no                       int64
section_name                    object
garment_group_no                 int64
garment_group_name              object
detail_desc                     object
dtype: object

# Merging datasets into one dataframe called df

In [49]:
df = customers.merge(
    transactions, on = "customer_id").merge(
    articles, on = "article_id")

df.head(2)

,customer_id,fashion_news,Active,club_member_status,fashion_news_frequency,age,postal_code,date,article_id,price,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0,0,ACTIVE,NONE,49,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...,2018-12-27,625548001,0.044051,...,Young Boy Outdoor,I,Children Sizes 134-170,4,Baby/Children,45,Kids Outerwear,1007,Outdoor,"Padded jacket with a detachable hood, stand-up..."
1,0209bfc5582b5c855952e5b43dc9c17bef921ca0f46b42...,0,0,ACTIVE,NONE,44,9a3af8c983d871be8b48db95a47f7c80b4961b293901c2...,2018-10-03,625548001,0.061000,...,Young Boy Outdoor,I,Children Sizes 134-170,4,Baby/Children,45,Kids Outerwear,1007,Outdoor,"Padded jacket with a detachable hood, stand-up..."


In [66]:
df.dtypes

customer_id                             object
fashion_news                             int64
Active                                   int64
club_member_status                      object
fashion_news_frequency                  object
age                                      int64
postal_code                             object
date                            datetime64[ns]
article_id                               int64
price                                  float64
sales_channel_id                         int64
product_code                             int64
prod_name                               object
product_type_no                          int64
product_type_name                       object
product_group_name                      object
graphical_appearance_no                  int64
graphical_appearance_name               object
colour_group_code                        int64
colour_group_name                       object
perceived_colour_value_id                int64
perceived_col

# Dropping irrelevant columns and splitting date

In [50]:
df = df.drop(
    ["postal_code", "detail_desc", "prod_name"],
    axis = 1)

df.head()

,customer_id,fashion_news,Active,club_member_status,fashion_news_frequency,age,date,article_id,price,sales_channel_id,...,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0,0,ACTIVE,NONE,49,2018-12-27,625548001,0.044051,1,...,8852,Young Boy Outdoor,I,Children Sizes 134-170,4,Baby/Children,45,Kids Outerwear,1007,Outdoor
1,0209bfc5582b5c855952e5b43dc9c17bef921ca0f46b42...,0,0,ACTIVE,NONE,44,2018-10-03,625548001,0.061000,1,...,8852,Young Boy Outdoor,I,Children Sizes 134-170,4,Baby/Children,45,Kids Outerwear,1007,Outdoor
2,05d20eff33ec820d4330ee25cce4d0c35e8a305137caf0...,1,1,ACTIVE,Regularly,41,2019-12-17,625548001,0.033881,2,...,8852,Young Boy Outdoor,I,Children Sizes 134-170,4,Baby/Children,45,Kids Outerwear,1007,Outdoor
3,05fa492d5d182adb9dca7cc20be545d2dc8990606d2197...,1,0,ACTIVE,Regularly,41,2018-11-22,625548001,0.050831,1,...,8852,Young Boy Outdoor,I,Children Sizes 134-170,4,Baby/Children,45,Kids Outerwear,1007,Outdoor
4,060de711fa176fb5249bb39f0eceea040bde903f8b0434...,0,0,ACTIVE,NONE,45,2018-09-24,625548001,0.067780,2,...,8852,Young Boy Outdoor,I,Children Sizes 134-170,4,Baby/Children,45,Kids Outerwear,1007,Outdoor


In [52]:
# Changing date to monthly date
df["kpi_date"] = pd.to_datetime(df["date"]).dt.to_period('M').dt.to_timestamp()

In [69]:
df.dtypes

customer_id                             object
fashion_news                             int64
Active                                   int64
club_member_status                      object
fashion_news_frequency                  object
age                                      int64
date                            datetime64[ns]
article_id                               int64
price                                  float64
sales_channel_id                         int64
product_code                             int64
product_type_no                          int64
product_type_name                       object
product_group_name                      object
graphical_appearance_no                  int64
graphical_appearance_name               object
colour_group_code                        int64
colour_group_name                       object
perceived_colour_value_id                int64
perceived_colour_value_name             object
perceived_colour_master_id               int64
perceived_col

# 1. Sales per Month and Year

In [25]:
sg = df.groupby([ "kpi_date",'sales_channel_id'])["price"].sum().reset_index()
sg["sales_growth"] = sg.groupby("sales_channel_id")["price"].pct_change()
sg = sg.rename(columns={"price": "revenue"})
sg.head(10)

,kpi_date,sales_channel_id,revenue,sales_growth
0,2018-09-01,1,5411.885186,NaN
1,2018-09-01,2,12343.754407,NaN
2,2018-10-01,1,11335.658407,1.094586
3,2018-10-01,2,30249.158203,1.450564
4,2018-11-01,1,9533.444797,-0.158986
5,2018-11-01,2,29467.543593,-0.025839
6,2018-12-01,1,10107.900695,0.060257
7,2018-12-01,2,22424.614017,-0.239006
8,2019-01-01,1,7031.654322,-0.304341
9,2019-01-01,2,26418.327153,0.178095


In [26]:
sg.to_sql(con = conn, name = "kpi_sales_growth", if_exists = "replace",index=False)

49

# 2. Average Order Value per Month and Year

In [34]:
aov = df.groupby(['kpi_date','sales_channel_id'])["price"].mean().reset_index()
aov.head(10)

,kpi_date,sales_channel_id,price
0,2018-09-01,1,5411.885186
1,2018-09-01,2,12343.754407
2,2018-10-01,1,11335.658407
3,2018-10-01,2,30249.158203
4,2018-11-01,1,9533.444797
5,2018-11-01,2,29467.543593
6,2018-12-01,1,10107.900695
7,2018-12-01,2,22424.614017
8,2019-01-01,1,7031.654322
9,2019-01-01,2,26418.327153


In [73]:
aov.to_sql(con = conn, name = "kpi_average_order_value", if_exists = "replace", index=False)

49

# 3. Fashion News Effectiveness

In [40]:
fne = df.groupby(['kpi_date', 'fashion_news'])['price'].sum().reset_index()
fne['revenue_normalized'] = fne.groupby('kpi_date')['price'].transform(lambda x: x.div(x.sum()).mul(100))
fne = fne.rename(columns={"price": "revenue"})

fne.head(10)


,kpi_date,fashion_news,revenue,revenue_normalized
0,2018-09-01,0,10422.503458,58.699679
1,2018-09-01,1,7333.136136,41.300321
2,2018-10-01,0,24374.214695,58.613255
3,2018-10-01,1,17210.601915,41.386745
4,2018-11-01,0,22793.816797,58.444203
5,2018-11-01,1,16207.171593,41.555797
6,2018-12-01,0,18932.289661,58.194978
7,2018-12-01,1,13600.225051,41.805022
8,2019-01-01,0,19443.761220,58.127868
9,2019-01-01,1,14006.220254,41.872132


In [41]:
fne.to_sql(con = conn, name = "kpi_fashion_news_effectiveness", if_exists = "replace", index=False)

50

# 4. Conversion Rate per Month and Year

In [81]:
cr = df.groupby(['kpi_date', 'sales_channel_id'])['customer_id'].nunique() / df.groupby(['kpi_date'])['customer_id'].nunique()
cr = cr.rename('conversion_rate').reset_index()
cr.head(10)


,kpi_date,sales_channel_id,conversion_rate
0,2018-09-01,1,0.497755
1,2018-09-01,2,0.557525
2,2018-10-01,1,0.479436
3,2018-10-01,2,0.621461
4,2018-11-01,1,0.442016
5,2018-11-01,2,0.658634
6,2018-12-01,1,0.532990
7,2018-12-01,2,0.569277
8,2019-01-01,1,0.430015
9,2019-01-01,2,0.662294


In [82]:
cr.to_sql(con = conn, name = "kpi_conversion_rate", if_exists = "replace",index=False)

49

# 5. Intentory Turnover

In [45]:
it = df.groupby(['kpi_date','sales_channel_id'])["product_code"].count() / df.groupby(['kpi_date'])["product_code"].nunique()
it = it.reset_index()
it = it.rename(columns={"product_code": "inventory_turnover"})
it.head(10)

,kpi_date,sales_channel_id,inventory_turnover
0,2018-09-01,1,15.392828
1,2018-09-01,2,28.766055
2,2018-10-01,1,24.505475
3,2018-10-01,2,56.430740
4,2018-11-01,1,21.510935
5,2018-11-01,2,54.410373
6,2018-12-01,1,25.671967
7,2018-12-01,2,40.992398
8,2019-01-01,1,19.552436
9,2019-01-01,2,55.138449


In [46]:
it.to_sql(con = conn, name = "kpi_inventory_turnover", if_exists = "replace",index=False)

49

# 6. Product Sales per Month and Year

In [54]:
ps = df.groupby(['kpi_date', 'product_type_name'])['price'].count().reset_index()
ps.head(10)

,kpi_date,product_type_name,price
0,2018-09-01,Alice band,20
1,2018-09-01,Bag,4519
2,2018-09-01,Ballerinas,857
3,2018-09-01,Beanie,4
4,2018-09-01,Belt,3234
5,2018-09-01,Bikini top,4881
6,2018-09-01,Blazer,8443
7,2018-09-01,Blouse,29229
8,2018-09-01,Bodysuit,2011
9,2018-09-01,Bootie,136


In [92]:
ps.to_sql(con = conn, name = "kpi_product_sales", if_exists = "replace", index=False)

2714

# 7. Product Sales (%) per Month and Year

In [93]:
psp = df.groupby(['kpi_date'])['product_type_name'].value_counts(normalize=True) * 100
psp = psp.rename('product_type_name_percentage').reset_index()
psp.head(10)

,kpi_date,product_type_name,product_type_name_percentage
0,2018-09-01,Sweater,18.664337
1,2018-09-01,Trousers,16.812716
2,2018-09-01,T-shirt,6.427966
3,2018-09-01,Dress,5.465419
4,2018-09-01,Blouse,4.914287
5,2018-09-01,Top,3.913742
6,2018-09-01,Bra,3.149253
7,2018-09-01,Shirt,3.127732
8,2018-09-01,Leggings/Tights,3.082001
9,2018-09-01,Underwear bottom,2.788108


In [94]:
psp.to_sql(con = conn, name = "kpi_product_sales_perc", if_exists = "replace", index=False)

2714

# 8. Purchase Frequency per Month and Year

In [96]:
# Calculate purchase frequency per month and year
pf = df.groupby(['kpi_date', 'customer_id'])['article_id'].count().mean(level=[0, 1]).reset_index()
pf.rename(columns={'article_id': 'purchase_frequency'}, inplace=True)

pf.head(20)

/var/folders/5f/06bzwqjd1kl3lkb5zlxrzcp00000gn/T/ipykernel_75933/1583856561.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  pf = df.groupby(['kpi_date', 'customer_id'])['article_id'].count().mean(level=[0, 1]).reset_index()


,kpi_date,customer_id,purchase_frequency
0,2018-09-01,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,8.0
1,2018-09-01,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,3.0
2,2018-09-01,00007d2de826758b65a93dd24ce629ed66842531df6699...,23.0
3,2018-09-01,0000f1c71aafe5963c3d195cf273f7bfd50bbf17761c91...,1.0
4,2018-09-01,00015c1a121e08bbd2552c15fbbb6e6b19d3bf8f7b6a3d...,1.0
5,2018-09-01,00018385675844f7a6babbed41b5655b5727fb16483b6e...,1.0
6,2018-09-01,0001f8cef6b9702d54abf66fd89eb21014bf98567065a9...,1.0
7,2018-09-01,00022754ec18c5e53757eea8b281632a5c4a499368ecc5...,1.0
8,2018-09-01,00025f8226be50dcab09402a2cacd520a99e112fe01fdd...,3.0
9,2018-09-01,0002b9088196900626214bcdc6d5f3d85f38fb462a4806...,1.0


In [97]:
pf.to_sql(con = conn, name = "kpi_purchase_frequency", if_exists = "replace", index=False)

6230341

# 9. Customer Retention Rate per Month and Year

In [98]:
crr = df.groupby(['kpi_date']) \
        .apply(lambda x: x[x['Active'] == 1]['customer_id'].nunique() / x['customer_id'].nunique()) \
        .reset_index(name='customer_retention_rate')
crr.head(15)

,kpi_date,customer_retention_rate
0,2018-09-01,0.401675
1,2018-10-01,0.399526
2,2018-11-01,0.400249
3,2018-12-01,0.406850
4,2019-01-01,0.412653
5,2019-02-01,0.416692
6,2019-03-01,0.420603
7,2019-04-01,0.411082
8,2019-05-01,0.408508
9,2019-06-01,0.407959


In [99]:
crr.to_sql(con = conn, name = "kpi_customer_retention", if_exists = "replace", index=False)

25

# 10. Fashion News Frequency per Month and Year

In [44]:
fnsr = (df.groupby(['kpi_date'])['fashion_news_frequency'].value_counts(normalize=True) * 100).round(2)
fnsr = fnsr.rename('fashion_news_frequency_percentage').reset_index()
fnsr.head(50)


,kpi_date,fashion_news_frequency,fashion_news_frequency_percentage
0,2018-09-01,NONE,58.70
1,2018-09-01,Regularly,41.23
2,2018-09-01,Monthly,0.06
3,2018-10-01,NONE,58.38
4,2018-10-01,Regularly,41.55
5,2018-10-01,Monthly,0.07
6,2018-11-01,NONE,58.31
7,2018-11-01,Regularly,41.63
8,2018-11-01,Monthly,0.06
9,2018-12-01,NONE,57.62


In [43]:
fnsr.to_sql(con = conn, name = "kpi_fashion_news_frequency", if_exists = "replace", index=False)

75